# Training

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd().parent
sys.path.append(str(ROOT))
from configs.path_config import EXTRACTED_DATA_DIR, MODEL_DIR
from configs.model_config import INPUT_FEATURES, OUTPUT_FEATURES, PARAMS
from src.processing import dataset
from models import lstm_model
from src import utils

#### Create the train data loader

In [ ]:
folder_path = EXTRACTED_DATA_DIR / 'group_alvbrodel_shifted'
data = dataset.StrainDataset(
    folder_path, 
    INPUT_FEATURES, 
    OUTPUT_FEATURES, 
    sequence_length=PARAMS['sequence_length'], 
    batch_size=PARAMS['batch_size'],
    test_size=PARAMS['test_size'], 
    start_idx=0)

train_loader = data.train_dataloader

#### Train the model

In [ ]:
# Input and output features
input_feature_names = data.input_feature_names  # List of input feature names
output_feature_names = data.output_feature_names  # List of output feature names
print(f"Input feature names: {input_feature_names}")
print(f"Output feature names: {output_feature_names}")
input_dim = data.input_feature_count  # Number of input features
output_dim = data.output_feature_count  # Number of output features
print(f"Input dimension: {input_dim}")
print(f"Output dimension: {output_dim}")

# Model parameters from path_config.py
hidden_dim = PARAMS['hidden_dim']
num_layers = PARAMS['num_layers']
num_epochs = PARAMS['num_epochs']
learning_rate = PARAMS['learning_rate']
dropout = PARAMS['dropout']

# Create the model and train it
model = lstm_model.LSTMModel(input_dim, input_dim, hidden_dim, num_layers, dropout)
print(model)

losses, prediction = lstm_model.training_loop(
    model=model,
    train_loader=train_loader,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    models_folder=MODEL_DIR,
    model_subname='all',
    input_features=INPUT_FEATURES,
    output_features=OUTPUT_FEATURES, 
    input_feature_names=input_feature_names,
    output_feature_names=output_feature_names,
)

#### Plot the epoch losses

In [ ]:
utils.plot_epochs_loss(num_epochs, losses)